# 🏠 Multi-Tenant Setup

> The central registry for multi-tenant SaaS applications - managing users, tenants, and access control.

---

In [ ]:
#| default_exp db_host

In [ ]:
#| export

from fastsql import *
from fastcore.utils import *
import uuid
import os
import urllib.parse
from datetime import datetime
from nbdev.showdoc import show_doc


## 🎯 Overview

The **Host Database** is the backbone of a multi-tenant architecture. It answers the critical questions:

| Question | Model | Purpose |
|----------|-------|---------|
| 👤 **Who is this person?** | `GlobalUser` | Identity & authentication |
| 🏢 **Where is their data?** | `TenantCatalog` | Database routing |
| 🔑 **What can they access?** | `Membership` | Access control |
| 💳 **Are they paying?** | `Subscription` | Billing status |
| 📋 **What happened?** | `HostAuditLog` | Security audit trail |
| ⚙️ **Background work?** | `SystemJob` | Async operations |

---

## 🏗️ Architecture

```
                    ┌─────────────────────────────────────┐
                    │         🏠 HOST DATABASE            │
                    │  (Single source of truth)           │
                    ├─────────────────────────────────────┤
                    │  👤 GlobalUser    → Identity        │
                    │  🏢 TenantCatalog → DB Routing      │
                    │  🔑 Membership    → Access Control  │
                    │  💳 Subscription  → Billing         │
                    └───────────┬─────────────────────────┘
                                │
            ┌───────────────────┼───────────────────┐
            ▼                   ▼                   ▼
    ┌───────────────┐   ┌───────────────┐   ┌───────────────┐
    │ 🗄️ Tenant A   │   │ 🗄️ Tenant B   │   │ 🗄️ Tenant C   │
    │   Database    │   │   Database    │   │   Database    │
    │  (isolated)   │   │  (isolated)   │   │  (isolated)   │
    └───────────────┘   └───────────────┘   └───────────────┘
```

**Key Principle:** User authenticates once → Host routes to correct tenant → Tenant data is isolated

## 🛠️ Utilities

Helper functions used throughout the host database operations.

In [ ]:
#| export

def timestamp(): return datetime.utcnow().isoformat()
def gen_id(): return uuid.uuid4().hex

In [ ]:
#| export

def get_db_uri(db) -> str:
    """
    Extract SQLAlchemy connection URI from a Database object.
    
    Safely renders the URL with the actual password (required for
    utilities like `map_and_upsert` that create new connections).
    
    **Why this is needed:**
    - `str(db.conn.engine.url)` masks the password as `***`
    - This causes "password authentication failed" errors
    - `render_as_string(hide_password=False)` reveals the actual password
    
    | Parameter | Description |
    |-----------|-------------|
    | `db` | fastsql/minidataapi Database object or HostDatabase instance |
    
    **Returns:** Full connection URI string with password
    
    **Example:**
    ```python
    from fh_saas.db_host import get_db_uri, HostDatabase
    from fh_saas.utils_polars_mapper import map_and_upsert
    
    host_db = HostDatabase.from_env()
    db_uri = get_db_uri(host_db.db)
    
    # Now safe to use with map_and_upsert
    map_and_upsert(df, 'my_table', 'id', db_uri)
    ```
    """
    import logging
    logger = logging.getLogger(__name__)
    
    # Handle both Database objects and HostDatabase wrapper
    if hasattr(db, 'db'):
        # It's a HostDatabase wrapper, get the inner Database
        engine = db.db.conn.engine
    elif hasattr(db, 'conn'):
        # It's a raw Database object
        engine = db.conn.engine
    elif hasattr(db, 'engine'):
        # It's something with an engine directly
        engine = db.engine
    else:
        raise TypeError(f"Cannot extract URI from {type(db)}. Expected Database or HostDatabase.")
    
    db_uri = engine.url.render_as_string(hide_password=False)
    logger.info(f"Extracted database URI for {engine.url.database}")
    
    return db_uri

In [ ]:
show_doc(get_db_uri)

---

### get_db_uri

>      get_db_uri (db)

*Extract SQLAlchemy connection URI from a Database object.*

Safely renders the URL with the actual password (required for
utilities like `map_and_upsert` that create new connections).

**Why this is needed:**
- `str(db.conn.engine.url)` masks the password as `***`
- This causes "password authentication failed" errors
- `render_as_string(hide_password=False)` reveals the actual password

| Parameter | Description |
|-----------|-------------|
| `db` | fastsql/minidataapi Database object or HostDatabase instance |

**Returns:** Full connection URI string with password

**Example:**
```python
from fh_saas.db_host import get_db_uri, HostDatabase
from fh_saas.utils_polars_mapper import map_and_upsert

host_db = HostDatabase.from_env()
db_uri = get_db_uri(host_db.db)

# Now safe to use with map_and_upsert
map_and_upsert(df, 'my_table', 'id', db_uri)
```

| Function | Description |
|----------|-------------|
| `timestamp()` | 🕐 Returns current UTC time in ISO format |
| `gen_id()` | 🆔 Generates a unique 32-character hex ID |
| `get_db_uri(db)` | 🔗 Extract full connection URI with password from Database object |

---

## 📦 Core Models

These dataclasses define the host database schema. Each model maps to a table with the `core_` or `sys_` prefix.

In [ ]:
#| export

class GlobalUser:
    """Identity: Who is this person?"""
    id: str; email: str; oauth_id: str
    password_hash: str = None; stripe_cust_id: str = None
    is_sys_admin: bool = False; created_at: str; last_login: str = None

class TenantCatalog:
    """Registry: Where is the database?"""
    id: str; name: str; db_url: str
    is_active: bool = True; plan_tier: str = "free"; created_at: str

class Membership:
    """Router: Which tenants can they access?"""
    id: str; user_id: str; tenant_id: str; profile_id: str
    meta_attributes: str = None; role: str = "member"
    is_active: bool = True; created_at: str

class Subscription:
    """Billing: Are they allowed to use the app?"""
    id: str; tenant_id: str
    stripe_sub_id: str; stripe_cust_id: str
    plan_tier: str; status: str
    current_period_end: str; cancel_at_period_end: bool = False
    payment_type: str = "subscription"  # 'subscription' or 'one_time'
    amount_cents: int = None  # For one-time payments
    product_name: str = None  # Product/plan description
    trial_end: str = None  # Trial end timestamp (ISO format)
    created_at: str = None  # Record creation timestamp

class HostAuditLog:
    """Security: Who changed the system?"""
    id: str; actor_user_id: str; event_type: str
    target_id: str = None; details: str = None; ip_address: str = None
    created_at: str

class SystemJob:
    """Maintenance: Provisioning & Cleanups"""
    id: str; job_type: str; status: str
    payload: str = None; error_log: str = None
    created_at: str; completed_at: str = None

class PricingPlan:
    """Pricing: Available subscription tiers with Stripe price IDs.
    
    Stores pricing configuration in database for admin-configurable tiers.
    Each plan can have monthly and/or yearly billing intervals.
    
    Attributes:
        id: Unique plan identifier (e.g., 'basic', 'pro', 'enterprise')
        name: Display name for UI (e.g., 'Basic Plan')
        description: Plan description for pricing page
        stripe_price_monthly: Stripe Price ID for monthly billing (price_xxx)
        stripe_price_yearly: Stripe Price ID for yearly billing (price_xxx)
        amount_monthly: Monthly price in cents (for display, e.g., 799 = $7.99)
        amount_yearly: Yearly price in cents (for display, e.g., 7900 = $79.00)
        currency: ISO currency code (default: 'usd')
        trial_days: Free trial period in days (default: 30)
        features: JSON array of feature keys enabled for this plan
        tier_level: Numeric level for feature gating (higher = more access)
        is_active: Whether plan is available for new subscriptions
        sort_order: Display order on pricing page
        created_at: Record creation timestamp
    
    Example:
        >>> plan = PricingPlan(
        ...     id='pro',
        ...     name='Pro Plan',
        ...     stripe_price_monthly='price_1234',
        ...     stripe_price_yearly='price_5678',
        ...     amount_monthly=1999,
        ...     amount_yearly=19900,
        ...     tier_level=2,
        ...     features='["api_access", "exports", "priority_support"]',
        ... )
    """
    id: str
    name: str
    description: str = None
    stripe_price_monthly: str = None
    stripe_price_yearly: str = None
    amount_monthly: int = None
    amount_yearly: int = None
    currency: str = "usd"
    trial_days: int = 30
    features: str = None  # JSON array
    tier_level: int = 1
    is_active: bool = True
    sort_order: int = 0
    created_at: str = None

### 📝 Model Details

| Model | Table Name | Primary Key | Description |
|-------|------------|-------------|-------------|
| `GlobalUser` | `core_users` | `id` | OAuth identity, email, optional password hash, Stripe customer ID |
| `TenantCatalog` | `core_tenants` | `id` | Maps tenant ID to database URL, tracks plan tier and status |
| `Membership` | `core_memberships` | `id` | Links users to tenants with roles (`owner`, `admin`, `member`) |
| `Subscription` | `core_subscriptions` | `id` | Stripe subscription state for billing enforcement |
| `PricingPlan` | `core_pricing_plans` | `id` | Available subscription tiers with Stripe price IDs |
| `HostAuditLog` | `sys_audit_logs` | `id` | Immutable security log for compliance |
| `SystemJob` | `sys_jobs` | `id` | Background task queue for provisioning, cleanup |

---

## 🔌 HostDatabase Singleton

The `HostDatabase` class provides a **singleton** connection manager for the host database. 

✅ **Why Singleton?**
- Single connection pool shared across the application
- Consistent transaction management
- Easy dependency injection for testing

```
┌─────────────────────────────────────────────────────┐
│                  Application Start                   │
└─────────────────────┬───────────────────────────────┘
                      ▼
         ┌────────────────────────┐
         │  HostDatabase.from_env() │  ← Reads DB_* env vars
         └────────────┬───────────┘
                      ▼
         ┌────────────────────────┐
         │   Creates tables if    │
         │   they don't exist     │
         └────────────┬───────────┘
                      ▼
         ┌────────────────────────┐
         │  Returns singleton     │  ← Same instance everywhere
         │  instance              │
         └────────────────────────┘
```

In [ ]:
#| export

class HostDatabase:
    """Singleton connection manager for the host database."""
    _instance = None
    
    def __new__(cls, db_url: str = None):
        """Singleton pattern - only one instance per application."""
        if cls._instance is None:
            cls._instance = super().__new__(cls)
            cls._instance._initialized = False
        return cls._instance
    
    def __init__(self, db_url: str = None):
        """Initialize database connection and create table objects."""
        if self._initialized:
            return
        
        if db_url is None:
            raise ValueError("db_url required for first initialization")
        
        # Create database connection
        self.db = Database(db_url)
        
        # Create table objects for host schema
        self.global_users = self.db.create(GlobalUser, name="core_users", pk='id')
        self.tenant_catalogs = self.db.create(TenantCatalog, name="core_tenants", pk='id')
        self.memberships = self.db.create(Membership, name="core_memberships", pk='id')
        self.subscriptions = self.db.create(Subscription, name="core_subscriptions", pk='id')
        self.pricing_plans = self.db.create(PricingPlan, name="core_pricing_plans", pk='id')
        self.audit_logs = self.db.create(HostAuditLog, name="sys_audit_logs", pk='id')
        self.system_jobs = self.db.create(SystemJob, name="sys_jobs", pk='id')
        
        self._initialized = True
    
    @property
    def engine(self):
        """Get the underlying SQLAlchemy engine."""
        return self.db.engine
    
    @classmethod
    def from_env(cls):
        """Create HostDatabase from DB_* environment variables."""
        DB_TYPE = os.getenv("DB_TYPE", "POSTGRESQL")
        DB_USER = os.getenv("DB_USER", "postgres")
        DB_PASS = os.getenv("DB_PASS", "")
        DB_HOST = os.getenv("DB_HOST", "localhost")
        DB_PORT = os.getenv("DB_PORT", "5432")
        DB_NAME = os.getenv("DB_NAME", "app_host")
        
        if DB_TYPE == "POSTGRESQL":
            if not DB_PASS:
                raise ValueError("DB_PASS is required for PostgreSQL")
            encoded_pass = urllib.parse.quote_plus(DB_PASS)
            db_url = f"postgresql://{DB_USER}:{encoded_pass}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
        else:
            db_url = f"sqlite:///{DB_NAME}.db"
        
        return cls(db_url)
    
    def commit(self):
        """Commit current transaction."""
        self.db.conn.commit()
    
    def rollback(self):
        """Rollback current transaction."""
        self.db.conn.rollback()
    
    def close(self):
        """Close database connection and dispose engine.
        
        Call this when shutting down or before reset_instance() in tests.
        """
        try:
            self.db.conn.close()
        except Exception:
            pass
        try:
            self.db.engine.dispose()
        except Exception:
            pass
        self._initialized = False
    
    @classmethod
    def reset_instance(cls):
        """Reset singleton instance (testing only).
        
        ⚠️ Call close() first to release database connections!
        """
        if cls._instance is not None:
            try:
                cls._instance.close()
            except Exception:
                pass
        cls._instance = None

### 🔧 Methods

| Method | Description |
|--------|-------------|
| `from_env()` | 🏭 Factory method - creates instance from environment variables |
| `commit()` | ✅ Commit the current database transaction |
| `rollback()` | ↩️ Rollback the current transaction on error |
| `reset_instance()` | 🧪 Reset singleton (testing only) |

#### 🌍 Environment Variables for `from_env()`

| Variable | Default | Description |
|----------|---------|-------------|
| `DB_TYPE` | `POSTGRESQL` | Database type (`POSTGRESQL` or `SQLITE`) |
| `DB_USER` | `postgres` | Database username |
| `DB_PASS` | *(required)* | Database password |
| `DB_HOST` | `localhost` | Database host |
| `DB_PORT` | `5432` | Database port |
| `DB_NAME` | `app_host` | Database name |

In [ ]:

show_doc(HostDatabase.from_env)

---

[source](https://github.com/abhisheksreesaila/fh-saas/blob/main/fh_saas/db_host.py#L100){target="_blank" style="float:right; font-size:smaller"}

### HostDatabase.from_env

>      HostDatabase.from_env ()

*Create HostDatabase from DB_* environment variables.*

In [ ]:
show_doc(HostDatabase.commit)

---

[source](https://github.com/abhisheksreesaila/fh-saas/blob/main/fh_saas/db_host.py#L119){target="_blank" style="float:right; font-size:smaller"}

### HostDatabase.commit

>      HostDatabase.commit ()

*Commit current transaction.*

In [ ]:
show_doc(HostDatabase.rollback)

---

[source](https://github.com/abhisheksreesaila/fh-saas/blob/main/fh_saas/db_host.py#L123){target="_blank" style="float:right; font-size:smaller"}

### HostDatabase.rollback

>      HostDatabase.rollback ()

*Rollback current transaction.*

In [ ]:
show_doc(HostDatabase.reset_instance)

---

[source](https://github.com/abhisheksreesaila/fh-saas/blob/main/fh_saas/db_host.py#L143){target="_blank" style="float:right; font-size:smaller"}

### HostDatabase.reset_instance

>      HostDatabase.reset_instance ()

*Reset singleton instance (testing only).*

⚠️ Call close() first to release database connections!

---

## 🚀 Quick Start

```python
from fh_saas.db_host import HostDatabase, GlobalUser, gen_id, timestamp

# Initialize singleton from environment
host_db = HostDatabase.from_env()

# Create a user
user = GlobalUser(
    id=gen_id(),
    email="user@example.com",
    oauth_id="google_123",
    created_at=timestamp()
)
host_db.global_users.insert(user)
host_db.commit()

# Query users
all_users = host_db.global_users()
```

💡 **Tip:** The singleton ensures you always get the same connection, so you can call `HostDatabase.from_env()` anywhere in your app.

In [ ]:
#| hide
import nbdev; 
nbdev.nbdev_export()